In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from state import get_state, queue, get_state_per_traffic_light
import os
import traci

In [2]:
from dotenv import load_dotenv
load_dotenv()


sumo_bin = os.getenv("SUMO")
#sumo_gui_bin = os.getenv("SUMO-GUI")
sumo_gui_bin = '/Library/Frameworks/EclipseSUMO.framework/Versions/1.22.0/EclipseSUMO/bin/sumo-gui'
simulConfig = os.getenv("SIMUL-CONFIG2")


print(sumo_gui_bin)

/Library/Frameworks/EclipseSUMO.framework/Versions/1.22.0/EclipseSUMO/bin/sumo-gui


In [ ]:
import numpy as np

from tensorflow.keras import Sequential, layers
from tensorflow.random import set_seed
from collections import deque
from tensorflow import keras
from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [64]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action_one_tf_light = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

In [ ]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [64]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action_second_tf_light = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

In [ ]:
def epsilon_greedy_policy(state_tf_one, state_tf_two,epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs), np.random.randint(n_outputs)  # random action
    else:
        Q_values_tf_one = model_action_one_tf_light.predict(state_tf_one[np.newaxis], verbose=0)[0]
        Q_values_tf_two = model_action_second_tf_light.predict(state_tf_two[np.newaxis], verbose=0)[0]

        print(f"state : {state_tf_one, state_tf_two}")
        print(f"Q_values : {Q_values_tf_one, Q_values_tf_two}")
        return Q_values_tf_one.argmax(), Q_values_tf_two.argmax()  # optimal action according to the DQN

In [ ]:
def sample_experiences(batch_size, name_batch):
    indices = np.random.randint(len(name_batch), size=batch_size)
    batch = [name_batch[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [ ]:
batch_size = 32
discount_factor = 0.5
optimizer_one = keras.optimizers.Adam(learning_rate=0.05)
loss_fn = MeanSquaredError()

def training_step_tf_one(batch_size, model, name_batch):
    experiences = sample_experiences(batch_size, name_batch)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer_one.apply_gradients(zip(grads, model.trainable_variables))

In [ ]:
batch_size = 32
discount_factor = 0.5
optimizer_two = keras.optimizers.Adam(learning_rate=0.05)
loss_fn = MeanSquaredError()

def training_step_tf_two(batch_size, model, name_batch):
    experiences = sample_experiences(batch_size, name_batch)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer_two.apply_gradients(zip(grads, model.trainable_variables))

In [ ]:
replay_buffer_tf_one = deque(maxlen=10000)
replay_buffer_tf_two = deque(maxlen=10000)

epoch = 100






# PERIODE DE STABILITE

# traffic_scale = [0.1] * 100
# AUGMENTATION DE TRAFFIC
# traffic_scale.extend(np.linspace(0.1, 1, 400, dtype=float))
# OR
#traffic_scale = 0.2



for episode in range(epoch):
    #SEED & TRAFFIC SCALING
    seed_value = np.random.randint(1000)
    #seed_value = 41
    if traci.isLoaded():
        traci.close()


    sumoCmd = [sumo_bin, "-c", simulConfig, "--start", "--no-warnings", "--seed", str(seed_value), '--scale', str(np.random.uniform(0.1, 0.2))]

    traci.start(sumoCmd)





    lane_ids =  traci.lane.getIDList()
    # print(lane_ids[0])

    traffic_light_ids = traci.trafficlight.getIDList()

    # state = np.array(queue(lane_ids))



    state_tf_one = np.array(get_state_per_traffic_light(traffic_light_ids[0]))
    state_tf_two = np.array(get_state_per_traffic_light(traffic_light_ids[1]))
    action_tf_one=-1
    action_tf_two=-1

    # print(state)
    for step in range(80000): ## TO CHANGED
        epsilon = max(1 - episode / epoch, 0.01)

        if step%2000 == 0:
            #print(f"longeur du buffer :{len(replay_buffer)}")
            ########################################################################
            ##Calcul de la reward
            # next_state = np.array(queue(lane_ids))
            next_state_tf_one = np.array(get_state_per_traffic_light(traffic_light_ids[0]))
            next_state_tf_two = np.array(get_state_per_traffic_light(traffic_light_ids[1]))

            # reward = calculate_reward(values, reward, total_reward)[0]
            # reward=(np.sum(state)-np.sum(next_state))
            reward_tf_one = np.sum(state_tf_one)- np.sum(next_state_tf_one)
            reward_tf_two = np.sum(state_tf_two)- np.sum(next_state_tf_two)

            replay_buffer_tf_one.append((state_tf_one, action_tf_one, reward_tf_one, next_state_tf_one))
            replay_buffer_tf_two.append((state_tf_two, action_tf_two, reward_tf_two, next_state_tf_two))
            #########################################################################
            state_tf_one = next_state_tf_one
            state_tf_two = next_state_tf_two

            action_tf_one, action_tf_two = epsilon_greedy_policy(state_tf_one,state_tf_two, epsilon)

            #print("action", action)
            traci.trafficlight.setPhase(traffic_light_ids[0],2*action_tf_one)
            traci.trafficlight.setPhase(traffic_light_ids[1],2*action_tf_two)

            if len(replay_buffer_tf_one) >= batch_size*10:
                training_step_tf_one(batch_size, model_action_one_tf_light, replay_buffer_tf_one)
                training_step_tf_two(batch_size, model_action_second_tf_light, replay_buffer_tf_two)

                # new_weights = model_action.get_weights()
                # weights.append(new_weights)
                #print(f"Episode {episode}: new weights = {new_weights}")
            # else:
            #     print(f"Episode {episode}: pas assez de données dans le replay buffer.")
            # print(values)
            # if list_values:
            #    # print(f'list values {list_values[-1]}')


        traci.simulationStep()
    print(f'episode : {episode}')
    traci.close()


 Retrying in 1 seconds


Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 167 ACT 23 BUF 0)                  episode : 0
Step #800.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 186 ACT 18 BUF 0)                  
 Retrying in 1 seconds


Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 155 ACT 15 BUF 0)                  episode : 1
Step #800.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 173 ACT 17 BUF 0)                  
 Retrying in 1 seconds


state : (array([0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 0, 0, 0]), array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]))
Q_values : (array([ 0.40591666, -0.2982662 ,  0.02721965, -0.58382004], dtype=float32), array([ 0.16603233, -0.29710624,  0.1908916 , -0.10720957], dtype=float32))
Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 133 ACT 10 BUF 0)                  episode : 2
Step #800.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 147 ACT 14 BUF 0)                  
 Retrying in 1 seconds


state : (array([0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 3, 0]))
Q_values : (array([ 0.28484675,  0.25571233, -0.33647642, -0.5877462 ], dtype=float32), array([-1.0134888 ,  0.73849314,  0.46222103,  0.20917822], dtype=float32))
Step #720.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 146 ACT 13 BUF 0)                  episode : 3
Step #800.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 164 ACT 22 BUF 0)                  
 Retrying in 1 seconds


...

state : (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
Q_values : (array([-3.7248924, -3.963139 , -4.3523464, -3.6097274], dtype=float32), array([-3.5071268, -3.7479496, -3.1112764, -4.3875027], dtype=float32))
state : (array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0

state : (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
Q_values : (array([-3.630094 , -3.9060779, -4.4474144, -2.981646 ], dtype=float32), array([-3.147949 , -4.421131 , -3.9598808, -4.821892 ], dtype=float32))
state : (array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0

state : (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
Q_values : (array([-3.8203232, -3.8782403, -4.077229 , -3.7761827], dtype=float32), array([-3.933816 , -3.4193625, -3.3572998, -4.4545403], dtype=float32))
state : (array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0

state : (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
Q_values : (array([-3.9429917, -4.445177 , -4.3826466, -4.057239 ], dtype=float32), array([-4.293975 , -4.090809 , -3.6514335, -4.4625626], dtype=float32))
state : (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0

state : (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
Q_values : (array([-4.2233467, -3.1639442, -3.8458726, -3.7361538], dtype=float32), array([-3.693951 , -3.5150208, -3.9284217, -4.39021  ], dtype=float32))
state : (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0

In [ ]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
# replay_buffer = deque(maxlen=2000)

#SEED é SCALE
seed_value = np.random.randint(1000)
#seed_value = 41
traffic_scale = 0.5


#LAUNCH
sumoCmd = [sumo_gui_bin, "-c", simulConfig, "--start", "--seed", str(seed_value), '--scale', str(traffic_scale), '--gui-settings-file', 'full_state_sumo_gui_settings.xml']

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
# print(lane_ids[0])
# for lane in lane_ids:
#     print(traci.lane.getLastStepVehicleNumber(lane))
# north_lane = traci.lane.getLastStepVehicleNumber("N_0")
# south_lane = traci.lane.getLastStepVehicleNumber("S_0")
# east_lane = traci.lane.getLastStepVehicleNumber("E_0")
# west_lane = traci.lane.getLastStepVehicleNumber("W_0")
trafic_light_ids = traci.trafficlight.getIDList()


state_tf_one=np.array(get_state_per_traffic_light(traffic_light_ids[0]))
state_tf_two = np.array(get_state_per_traffic_light(traffic_light_ids[1]))
action_tf_one=-1
action_tf_two=-1# print(state)


for step in range(100000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)

    if step%2000 == 0:
        state_tf_one=np.array(get_state_per_traffic_light(traffic_light_ids[0]))
        state_tf_two = np.array(get_state_per_traffic_light(traffic_light_ids[1]))
        action_tf_one, action_tf_two = epsilon_greedy_policy(state_tf_one,state_tf_two, epsilon)
        #action = np.random.randint(3)
        print("action", action_tf_one, action_tf_two)
        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu))

        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu)[0].phases[action])
        traci.trafficlight.setPhase(traffic_light_ids[0],2*action_tf_one)
        traci.trafficlight.setPhase(traffic_light_ids[1],2*action_tf_two)
    traci.simulationStep()

traci.close()


 Retrying in 1 seconds
state : (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
Q_values : (array([-4.3187275, -3.8242278, -3.9024222, -3.9492745], dtype=float32), array([-3.3459735, -4.0678873, -3.961885 , -3.8233736], dtype=float32))
action 1 0
state : (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 